In [12]:
#zadávání jednotlivých proměnných
#vytvoření časové značky, kde se zobrazí aktuální datum a čas 
from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
#vytvoření proměnné pro mapu, která bude screenshot z mapového pole z připojené mapové komponenty a propsán jako fotka do té proměnné
    # tady bude proměnná mapa jako screenshot z mapové komponenty, kde proběhl výběr území 
    
    # proměnná výběru tématických vrstev 

    # proměnná hodnoty

    # proměnná opatření 

    # proměnná předpokládaného efektu opatření případně co by hrozilo kdyby se opatření nepřijala



In [13]:
# vytvoření html šablony, kde se budou propisovat jednotlivé proměnné
html = """
<!DOCTYPE html>
<html>
<head>
    <title>Georeport zranitelnosti krajiny</title>
			<meta charset='UTF-8'>
			<meta name='description' content='HTML stránka georeportu zranitelnosti krajiny'>
			<meta name='keywords' content='zranitelnost krajiny, georeport, ekologie'>
			<meta name='author' content='Adéla Petřeková'>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }}
        h1 {{
            background-color: #333;
            color: #fff;
            padding: 10px;
        }}
        p {{
            padding: 10px;
        }}
    </style>

</head>
<body>
    <h1>Georeport zranitelnosti krajiny</h1>
    <p align="right">Vytvořeno v: {timestamp}</p>
    <p>Tady bude další text a tabulky/obrázky.</p>
    

</body>
</html>
"""
#css nedavat moc jako samostatny soubor ale dat to do primo te sablony

In [14]:
#vytvoření HTML souboru ze šablony
with open("index.html", "w") as f:
    f.write(html.format(timestamp=timestamp))
#otevření html souboru
print("Generated index.html")
import webbrowser
webbrowser.open("index.html")



Generated index.html


True

In [ ]:
# output html as a PDF
# first, import the necessary modules

from weasyprint import HTML

In [ ]:
# then, convert the HTML to a PDF
HTML("index.html").write_pdf("index.pdf")

print("Generated index.pdf")

# api esri na vytvoření bodu, linie a polygonu
https://developers.arcgis.com/python/guide/add-a-point-line-and-polygon/ 
# tutorial esri
 https://developers.arcgis.com/documentation/mapping-apis-and-services/tutorials/

 co následně bude potřeba udelat: 
 - implementace mapového okna - esri api atd
 - napojení databáze s daty
 - jakmile bude mapové okno tak více css rozdělení do sloupce 
 

